In [1]:
# ----------------------------
# Step 1: Setup & Imports
# ----------------------------
import os
import sys
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Add project root to system path
sys.path.append("..")

from src.model_config import ModelConfig
from src.model_architecture import LeukemiaDetectionModel
from src.data_loader import DataLoader# ----------------------------
# Step 1: Setup & Imports
# ----------------------------
import os
import sys
import matplotlib.pyplot as plt
from pathlib import Path

try:
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
except ImportError as e:
    raise ImportError("TensorFlow is not installed. Please install it using 'pip install tensorflow' before running this script.") from e

# Add project root to system path
sys.path.append("..")

from src.model_config import ModelConfig
from src.model_architecture import LeukemiaDetectionModel
from src.data_loader import DataLoader
from src.data_augmentation import get_augmentation_pipeline
from src.utils import (
    setup_logging, print_system_info,
    plot_training_history, save_model_history,
    create_experiment_folder
)

# ----------------------------
# Step 2: Load Configurations
# ----------------------------
config = ModelConfig()
model_params = config.get_model_params()
training_params = config.get_training_params()
data_params = config.get_data_params()
paths = config.paths_config

# Logging and system info
setup_logging(log_level="INFO")
print_system_info()

# ----------------------------
# Step 3: Load Data
# ----------------------------
loader = DataLoader(
    dataset_path=data_params["dataset_path"],
    processed_path=data_params["processed_path"],
    image_size=tuple(data_params["image_size"]),
    seed=data_params["random_seed"]
)

train_ds, val_ds, test_ds, class_names = loader.get_tf_datasets(
    batch_size=data_params["batch_size"]
)

# Apply data augmentation to training set
augmenter = get_augmentation_pipeline(config.augmentation_config)
train_ds = train_ds.map(lambda x, y: (augmenter(x, training=True), y))

# ----------------------------
# Step 4: Setup Experiment Paths
# ----------------------------
experiment_path = create_experiment_folder()
checkpoint_path = Path(paths["checkpoint_path"]) / "best_model.keras"
checkpoint_path.parent.mkdir(parents=True, exist_ok=True)

history_path = Path(experiment_path) / "history.json"
plot_path = Path(experiment_path) / "training_plot.png"

# ----------------------------
# Step 5: Build & Compile Model
# ----------------------------
model = LeukemiaDetectionModel(config).build_model()
model.summary()

model.compile(
    optimizer=training_params["optimizer"],
    loss=training_params["loss_function"],
    metrics=training_params["metrics"]
)

# ----------------------------
# Step 6: Define Callbacks
# ----------------------------
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=training_params["early_stopping_patience"],
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=training_params["reduce_lr_factor"],
        patience=training_params["reduce_lr_patience"],
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(checkpoint_path),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )
]

# ----------------------------
# Step 7: Train the Model
# ----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=training_params["epochs"],
    callbacks=callbacks
)

# ----------------------------
# Step 8: Save Results
# ----------------------------
save_model_history(history, str(history_path))

plot_training_history(
    {
        'epoch': list(range(1, len(history.history['loss']) + 1)),
        **history.history
    },
    save_path=str(plot_path)
)

# ----------------------------
# Step 9: Final Output
# ----------------------------
print(f"\n✅ Training complete.")
print(f"📦 Best model saved at: {checkpoint_path}")
print(f"📊 Training history saved at: {history_path}")
print(f"📈 Training plot saved at: {plot_path}")

from src.data_augmentation import get_augmentation_pipeline
from src.utils import (
    setup_logging, print_system_info,
    plot_training_history, save_model_history,
    create_experiment_folder
)

# ----------------------------
# Step 2: Load Configurations
# ----------------------------
config = ModelConfig()
model_params = config.get_model_params()
training_params = config.get_training_params()
data_params = config.get_data_params()
paths = config.paths

# Logging and system info
setup_logging(log_level="INFO")
print_system_info()

# ----------------------------
# Step 3: Load Data
# ----------------------------
loader = DataLoader(
    dataset_path=data_params["dataset_path"],
    processed_path=data_params["processed_path"],
    image_size=tuple(data_params["image_size"]),
    seed=data_params["random_seed"]
)

train_ds, val_ds, test_ds, class_names = loader.get_tf_datasets(
    batch_size=data_params["batch_size"]
)

# Apply data augmentation to training set
augmenter = get_augmentation_pipeline(config.augmentation_config)
train_ds = train_ds.map(lambda x, y: (augmenter(x, training=True), y))

# ----------------------------
# Step 4: Setup Experiment Paths
# ----------------------------
experiment_path = create_experiment_folder()
checkpoint_path = Path(paths["checkpoint_path"]) / "best_model.keras"
checkpoint_path.parent.mkdir(parents=True, exist_ok=True)

history_path = Path(experiment_path) / "history.json"
plot_path = Path(experiment_path) / "training_plot.png"

# ----------------------------
# Step 5: Build & Compile Model
# ----------------------------
model = LeukemiaDetectionModel(config).build_model()
model.summary()

model.compile(
    optimizer=training_params["optimizer"],
    loss=training_params["loss_function"],
    metrics=training_params["metrics"]
)

# ----------------------------
# Step 6: Define Callbacks
# ----------------------------
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=training_params["early_stopping_patience"],
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=training_params["reduce_lr_factor"],
        patience=training_params["reduce_lr_patience"],
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(checkpoint_path),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )
]

# ----------------------------
# Step 7: Train the Model
# ----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=training_params["epochs"],
    callbacks=callbacks
)

# ----------------------------
# Step 8: Save Results
# ----------------------------
save_model_history(history, str(history_path))

plot_training_history(
    {
        'epoch': list(range(1, len(history.history['loss']) + 1)),
        **history.history
    },
    save_path=str(plot_path)
)

# ----------------------------
# Step 9: Final Output
# ----------------------------
print(f"\n✅ Training complete.")
print(f"📦 Best model saved at: {checkpoint_path}")
print(f"📊 Training history saved at: {history_path}")
print(f"📈 Training plot saved at: {plot_path}")

ModuleNotFoundError: No module named 'utils'